In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import boxcox
from sqlalchemy import create_engine
from scipy.stats.mstats import winsorize
import warnings

warnings.filterwarnings('ignore')

In [ ]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'lifeexpectancy'
table_name = 'lifeexpectancy'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

life = pd.read_sql_query('select * from lifeexpectancy',con=engine)

In [ ]:
#Inspect dataframe
life.head(20)

1. Your goal in this challenge is to find the factors that affect the life expectancy. Specifically, you need to find out which factors increase the expected life in the countries and which factors decrease it.

2. Detect the problems with the data such as missing values and outliers. Are there any nonsense values that seem to be stemmed from the data collection? For the missing values, discuss which technique would be the most suitable one in filling out these values. Regarding the outliers, discuss their potential effects on your analysis and select an appropriate method to deal with them.

In [ ]:
#Inspect data
life.describe()

In [ ]:
#Inspect categorical data
life.describe(include=['O'])

In [ ]:
life.info()

In [ ]:
#Clean up leading/trailing spaces in column names
new_names= list(life.columns.str.strip())
life.columns = new_names

In [ ]:
#In WHO's 'about' section, thinness 1-19 years explained as thinness
#stats gathered regarding citizens 10-19 years old. Renaming to clarify.
life.rename(columns={'thinness  1-19 years': 'thinness 10-19 years'}, inplace=True)

In [ ]:
#Find max and min years to use for interpolation exceptions
print('Max Year:', max(life.Year), 'Min Year:', min(life.Year))

In [ ]:
#Time series data- best method is interpolation. 
if [life.Year== 2000]:
    life.fillna(method='bfill', inplace=True)
elif [life.Year ==2015]:
    life.fillna(method='ffill', inplace=True)    
else:
    life.interpolate(inplace=True)

In [ ]:
#Replace 0's with column mean
infant_death_mean= life.loc[life['infant deaths']!=0, 'infant deaths'].mean()
life['infant deaths']= life['infant deaths'].replace(0, infant_death_mean)

In [ ]:
#Check whether all nulls are filled
life.isnull().sum()*100/life.isnull().count()

In [ ]:
sns.set_style('whitegrid')

plt.hist(life['infant deaths'])
plt.title('Distribution of infant deaths variable')

Infant Deaths has a strong right skew- one way winsorization will help eliminate outliers.

In [ ]:
#Winsorize right skew
life['infant deaths']= winsorize(life['infant deaths'], (0, 0.10))

In [ ]:
#Display histogram distributions of next nine variables
plt.figure(figsize=(20, 20))

plt.subplot(3, 3, 1)
plt.hist(life['Adult Mortality'])
plt.title('Distribution of Adult Mortality')

plt.subplot(3, 3, 2)
plt.hist(life['Hepatitis B'])
plt.title('Distribution of Hepatitis B')

plt.subplot(3, 3, 3)
plt.hist(life['under-five deaths'])
plt.title('Distribution of Child Deaths Under 5')

plt.subplot(3, 3, 4)
plt.hist(life['Polio'])
plt.title('Distribution of Polio')

plt.subplot(3, 3, 5)
plt.hist(life['Measles'])
plt.title('Distribution of Measles')

plt.subplot(3, 3, 6)
plt.hist(life['Total expenditure'])
plt.title('Distribution of Total expenditure')

plt.subplot(3, 3, 7)
plt.hist(life['GDP'])
plt.title('Distribution of GDP')

plt.subplot(3, 3, 8)
plt.hist(life['BMI'])
plt.title('Distribution of BMI')

plt.subplot(3, 3, 9)
plt.hist(life['Diphtheria'])
plt.title('Distribution of Diphtheria')

plt.suptitle('Distributions of Variables Pt. 1')

Child Deaths, GDP, and Measles have some major outliers on the right- they will need to be log transformed. Polio and Diphtheria have left outliers, which we will one-way winsorize.

In [ ]:
#Log transform first variables, adding constants to make all values positive
life['under_five_deaths_log']= np.log(life['under-five deaths']+4.473)
life['GDP_log'] = np.log(life['GDP']+5.304)
life['Measles_log']= np.log(life['Measles']+5.835)

#Winsorizing Polio and Diphtheria
life['Polio']= winsorize(life['Polio'], (0.10, 0))
life['Diphtheria']= winsorize(life['Diphtheria'], (0.10, 0))
life['Adult Mortality']=winsorize(life['Adult Mortality'], (0, 0.05))

In [ ]:
plt.figure(figsize=(10, 15))

plt.subplot(2, 3, 1)
plt.hist(life['under_five_deaths_log'])
plt.title('U-5 Deaths log transformed')

plt.subplot(2, 3, 2)
plt.hist(life['GDP_log'])
plt.title('GDP log transformed')

plt.subplot(2, 3, 3)
plt.hist(life['Measles_log'])
plt.title('Measles log transformed')

plt.subplot(2, 3, 4)
plt.hist(life['Adult Mortality'])
plt.title('Adult Mortality log transformed')

plt.subplot(2, 3, 5)
plt.hist(life['Polio'])
plt.title('Polio winsorized')

plt.subplot(2, 3, 6)
plt.hist(life['Diphtheria'])
plt.title('Diphtheria winsorized')

plt.show()

In [ ]:
#Check remaining outliers using Tukey's method
q75, q25 = np.percentile(life['Adult Mortality'], [75 ,25])
iqr = q75 - q25

for threshold in np.arange(1,5,0.5):
    min_val = q25 - (iqr*threshold)
    max_val = q75 + (iqr*threshold)
    print("The score threshold is: {}".format(threshold))
    print("Number of outliers is: {}".format(
        len((np.where((life['Adult Mortality'] > max_val) 
                      | (life['Adult Mortality'] < min_val))[0]))
    ))

In [ ]:
#Conduct Box-Cox transformations for remaining variables
life['hepatitis_b_boxcox'], _= boxcox(life['Hepatitis B'])
life['total_expenditure_boxcox'], _= boxcox(life['Total expenditure'])
life['bmi_boxcox'], _= boxcox(life['BMI'])

In [ ]:
#Checking distributions round 2
plt.figure(figsize=(20, 20))

plt.subplot(3, 3, 1)
plt.hist(life['Life expectancy'])
plt.title('Distribution of Life Expectancy')

plt.subplot(3, 3, 2)
plt.hist(life['Alcohol'])
plt.title('Distribution of Alcohol')

plt.subplot(3, 3, 3)
plt.hist(life['percentage expenditure'])
plt.title('Distribution of percentage expenditure')

plt.subplot(3, 3, 4)
plt.hist(life['HIV/AIDS'])
plt.title('Distribution of HIV/AIDS')

plt.subplot(3, 3, 5)
plt.hist(life['Population'])
plt.title('Distribution of Population')

plt.subplot(3, 3, 6)
plt.hist(life['thinness 10-19 years'])
plt.title('Distribution of thinness  1-19 years')

plt.subplot(3, 3, 7)
plt.hist(life['thinness 5-9 years'])
plt.title('Distribution of thinness 5-9 years')

plt.subplot(3, 3, 8)
plt.hist(life['Income composition of resources'])
plt.title('Distribution of Income composition of resources')

plt.subplot(3, 3, 9)
plt.hist(life['Schooling'])
plt.title('Distribution of Schooling')

plt.suptitle('Distributions of Variables, Pt. 2')

percentage expenditure, HIV/AIDS, Alcohol, thinness(both), Population, all have right strong right skews that should be winsorized. 

In [ ]:
#Winsorizing 
life['percentage expenditure'] = winsorize(life['percentage expenditure'], (0, 0.10))
life['HIV/AIDS'] = winsorize(life['HIV/AIDS'], (0, 0.05))
life['thinness 10-19 years'] = winsorize(life['thinness 10-19 years'], (0, 0.05))
life['thinness 5-9 years'] = winsorize(life['thinness 5-9 years'], (0, 0.05))
life['Population'] = winsorize(life['Population'], (0, 0.10))
life['Alcohol'] = winsorize(life['Alcohol'], (0, 0.05))

In [ ]:
#Using Tukey's method, test winsorized versions for outliers

print('-----------')    
print('Outliers for percentage expenditure:')
print('-----------')
q75, q25 = np.percentile(life['percentage expenditure'], [75 ,25])
iqr = q75 - q25

for threshold in np.arange(1,5,0.5):
    min_val = q25 - (iqr*threshold)
    max_val = q75 + (iqr*threshold)
    print("The score threshold is: {}".format(threshold))
    print("Number of outliers is: {}".format(
        len((np.where((life['percentage expenditure'] > max_val) 
                      | (life['percentage expenditure'] < min_val))[0]))
    ))

print('-----------')    
print('Outliers for HIV/AIDS:')
print('-----------')
q75, q25 = np.percentile(life['HIV/AIDS'], [75 ,25])
iqr = q75 - q25

for threshold in np.arange(1,5,0.5):
    min_val = q25 - (iqr*threshold)
    max_val = q75 + (iqr*threshold)
    print("The score threshold is: {}".format(threshold))
    print("Number of outliers is: {}".format(
        len((np.where((life['HIV/AIDS'] > max_val) 
                      | (life['HIV/AIDS'] < min_val))[0]))
    ))

print('-----------')    
print('Outliers for thinness 10-19 years:')
print('-----------')
q75, q25 = np.percentile(life['thinness 10-19 years'], [75 ,25])
iqr = q75 - q25

for threshold in np.arange(1,5,0.5):
    min_val = q25 - (iqr*threshold)
    max_val = q75 + (iqr*threshold)
    print("The score threshold is: {}".format(threshold))
    print("Number of outliers is: {}".format(
        len((np.where((life['thinness 10-19 years'] > max_val) 
                      | (life['thinness 10-19 years'] < min_val))[0]))
    ))

print('-----------')    
print('Outliers for thinness 5-9 years:')
print('-----------')
q75, q25 = np.percentile(life['thinness 5-9 years'], [75 ,25])
iqr = q75 - q25

for threshold in np.arange(1,5,0.5):
    min_val = q25 - (iqr*threshold)
    max_val = q75 + (iqr*threshold)
    print("The score threshold is: {}".format(threshold))
    print("Number of outliers is: {}".format(
        len((np.where((life['thinness 5-9 years'] > max_val) 
                      | (life['thinness 5-9 years'] < min_val))[0]))
    ))
    
print('-----------')    
print('Outliers for Population:')
print('-----------')
q75, q25 = np.percentile(life['Population'], [75 ,25])
iqr = q75 - q25

for threshold in np.arange(1,5,0.5):
    min_val = q25 - (iqr*threshold)
    max_val = q75 + (iqr*threshold)
    print("The score threshold is: {}".format(threshold))
    print("Number of outliers is: {}".format(
        len((np.where((life['Population'] > max_val) 
                      | (life['Population'] < min_val))[0]))
    ))
    
print('-----------')    
print('Outliers for Alcohol:')
print('-----------')
q75, q25 = np.percentile(life['Alcohol'], [75 ,25])
iqr = q75 - q25

for threshold in np.arange(1,5,0.5):
    min_val = q25 - (iqr*threshold)
    max_val = q75 + (iqr*threshold)
    print("The score threshold is: {}".format(threshold))
    print("Number of outliers is: {}".format(
        len((np.where((life['Alcohol'] > max_val) 
                      | (life['Alcohol'] < min_val))[0]))
    ))

Winsorization had very little effect on eliminating outliers in percentage expenditure and HIV/AIDS, so we will box-cox transform those along with the remaining variables. 

In [ ]:
#Conduct Box-Cox transformations on remaining variables
life['income_composition_of_resources_boxcox'], _= boxcox(life['Income composition of resources']+0.001)
life['schooling_boxcox'], _= boxcox(life['Schooling']+0.001)
life['percentage expenditure'], _= boxcox(life['percentage expenditure']+0.01)
life['HIV/AIDS'], _= boxcox(life['HIV/AIDS']+0.01)

_Explore the data using univariate and multivariate exploration techniques. You should pay special attention to your target variable. In this regard, your focus should be on finding the relevant variables that may affect life expectancy._

In [ ]:
#Explore original variable correlations with Life Expectancy
plt.figure(figsize=(20, 20))

plt.subplot(3, 3, 1)
plt.scatter(life['Life expectancy'], life['Adult Mortality'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('Adult Mortality')
plt.title('Life expectancy vs. Adult Mortality')

plt.subplot(3, 3, 2)
plt.scatter(life['Life expectancy'], life['infant deaths'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('Infant Deaths')
plt.title('Life expectancy vs. infant deaths')

plt.subplot(3, 3, 3)
plt.scatter(life['Life expectancy'], life['Alcohol'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('Alcohol')
plt.title('Life expectancy vs. Alcohol')

plt.subplot(3, 3, 4)
plt.scatter(life['Life expectancy'], life['percentage expenditure'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('Percentage Expenditure')
plt.title('Life expectancy vs. Percentage Expenditure')

plt.subplot(3, 3, 5)
plt.scatter(life['Life expectancy'], life['Hepatitis B'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('Hepatitis B')
plt.title('Life expectancy vs. Hepatitis B')

plt.subplot(3, 3, 6)
plt.scatter(life['Life expectancy'], life['Measles'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('Measles')
plt.title('Life expectancy vs. Measles')

plt.subplot(3, 3, 7)
plt.scatter(life['Life expectancy'], life['BMI'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('BMI')
plt.title('Life expectancy vs. BMI')

plt.subplot(3, 3, 8)
plt.scatter(life['Life expectancy'], life['under-five deaths'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('Under Five Deaths')
plt.title('Life expectancy vs. Under Five Deaths')

plt.subplot(3, 3, 9)
plt.scatter(life['Life expectancy'], life['Polio'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('Polio')
plt.title('Life expectancy vs. Polio')

plt.tight_layout()
plt.show()

Adult Mortality, infant deaths, and under-five death rates all have a negative correlation as life expectancy increases. 

In [ ]:
#Compare original variables against Life expectancy
plt.figure(figsize=(20, 20))

plt.subplot(3, 3, 1)
plt.scatter(life['Life expectancy'], life['Total expenditure'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('Total expenditure')
plt.title('Life expectancy vs. Total expenditure')

plt.subplot(3, 3, 2)
plt.scatter(life['Life expectancy'], life['Diphtheria'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('Diphtheria')
plt.title('Life expectancy vs. Diphtheria')

plt.subplot(3, 3, 3)
plt.scatter(life['Life expectancy'], (life['HIV/AIDS']))
plt.xlabel('Life Expectancy(years)')
plt.ylabel('HIV/AIDS')
plt.title('Life expectancy vs. HIV/AIDS')

plt.subplot(3, 3, 4)
plt.scatter(life['Life expectancy'], life['GDP'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('GDP')
plt.title('Life expectancy vs. GDP')

plt.subplot(3, 3, 5)
plt.scatter(life['Life expectancy'], life['Population'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('Population')
plt.title('Life expectancy vs. Population')

plt.subplot(3, 3, 6)
plt.scatter(life['Life expectancy'], life['thinness 10-19 years'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('thinness 10-19 years')
plt.title('Life expectancy vs. thinness 10-19 years')

plt.subplot(3, 3, 7)
plt.scatter(life['Life expectancy'], life['thinness 5-9 years'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('thinness 5-9 years')
plt.title('Life expectancy vs. thinness 5-9 years')

plt.subplot(3, 3, 8)
plt.scatter(life['Life expectancy'], life['Income composition of resources'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('Income composition of resources')
plt.title('Life expectancy vs. Income composition of resources')

plt.subplot(3, 3, 9)
plt.scatter(life['Life expectancy'], life['Schooling'])
plt.xlabel('Life Expectancy(years)')
plt.ylabel('Schooling')
plt.title('Life expectancy vs. Schooling')

plt.tight_layout()
plt.show()

HIV/AIDS appears to have an exponential negative correlation as Expectancy increases, while GDP has an exponential positive correlation. Income composition of resources and Schooling have a positive, almost linear, correlation. 

In [ ]:
#Hot encode Status
pd.get_dummies(life['Status'], drop_first=True)
life= pd.concat([life, pd.get_dummies(life['Status'], drop_first=True)], axis=1)

In [ ]:
#Inspect heatmap of continuous variables
plt.figure(figsize=(30, 30))

life_corrmat= life.corr()

sns.heatmap(life_corrmat, square=True, annot=True, linewidth=1)

Of the original variables, income composition of resources and Schooling have the highest correlation with Life Expectancy, while Adult Mortality has the strongest negative correlation. Of the transformed variables, HIV/AIDS has a strong negative correlation and income composition of resources boxcox has an even stronger correlation.

In [ ]:
#Drop variables having weak correlation with life expectancy
life= life.drop(columns={'infant deaths', 'Hepatitis B', 'Measles', 'Measles_log', 'Diphtheria',
                         'hepatitis_b_boxcox', 'under-five deaths', 'Population', 
                         'Total expenditure', 'Developing', 'total_expenditure_boxcox', 'BMI',
                        'thinness 10-19 years', 'thinness 5-9 years', 'percentage expenditure',
                        'GDP', 'Alcohol', 'Income composition of resources', 'Schooling', 'schooling_boxcox'})

In [ ]:
#Standardize variables for PCA
life2= life.copy()
life2= life2.drop(columns={'Year', 'Country', 'Status'})

In [ ]:
X=StandardScaler().fit_transform(life2)
Xt=X.T
Cx=np.cov(Xt)

In [ ]:
eig_val_cov, eig_vec_cov= np.linalg.eig(Cx)

In [ ]:
plt.plot(eig_val_cov)
plt.title('Scree plot of Eigenvalues')
plt.show()
print(list(eig_val_cov))

The >1 rule and scree plot dictate we should keep 1 component.

In [ ]:
#PCA time
sklearn_pca = PCA(n_components=1)
life2["pca_1"] = sklearn_pca.fit_transform(X)
life2["pca_1"]

In [ ]:
#Examine 
print('The percentage of variance explained by each component in the PCA:', 
     list(eig_val_cov/sum(eig_val_cov)*100))

lst_eig_val_pcts= list(eig_val_cov/sum(eig_val_cov)*100)

print('The first component holds {} percent of the dataset variance.'.format(int(sum(lst_eig_val_pcts[:1]))))

In [ ]:
life2[['Life expectancy', 'pca_1']].corr()

The PCA generated has a correlation coefficient lower than -.8, so this may be unstable in future principal component estimations.

_In the feature engineering step, you need to select a suite of variables that you think would be ideal in the modeling phase. More concretely, you may discard some variables that are very correlated with the other ones or the variables that you think irrelevant with the life expectancy._

In [ ]:
print('These variables have a strong relationship with life expectancy, and will be best utilized in the modeling phase: \n {}'.format(list(life2.columns[:-1])))

_Summarize your findings. One of the most important skills of a data scientist is to convey ideas and findings to nontechnical people using understandable language. In this regard, one of the most effective ways to communicate your ideas is to do it using effective visualization._

In [ ]:
#Checking updated heatmap
plt.figure(figsize=(30, 30))

life_corrmat= life.corr()

sns.heatmap(life_corrmat, square=True, annot=True, linewidth=1)

This heatmap shows the correlation of the remaining variables to our target variable: Life expectancy. A score further away from 0 predicts a stronger correlation. HIV/AIDS and Income composition of resources appear to have the strongest correlation with life expectancy, so as rates of HIV/AIDS increase, life expectancy decreases, and as income composition of resources increase, as do life expectancies. Note that no values >0.8 or < -0.8 are remaining, as they will interfere with the next phases of modeling. 